# Initialization

In [ ]:
import os
import json

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models

# Dataset Initialization

In [ ]:
datapath = '../data/acousticbrainz-mediaeval_labels_part_a'

data_a = pd.read_csv(datapath + 'a', delimiter='\t')
data_b = pd.read_csv(datapath + 'b', delimiter='\t')

data = pd.concat([data_a, data_b], ignore_index=True)

# labels the songs that are rnb as is_rnb
data['is_rnb'] = data.filter(like='genre').apply(lambda x: x.astype(str).str.contains(r'R&B|rnb|r&b_soul|r\'n\'b', case=False, na=False)).any(axis=1).astype(int)

data.keys()

In [ ]:
# only need the lables for if it is rnb and song identifier
data_labeled = data[['recordingmbid', 'is_rnb']]
print(data_labeled['is_rnb'].value_counts())

In [ ]:
print(data.shape)
data.head()

# Feature Extraction
Extraction process from the Load_data notebook. For more details, please look there.

In [ ]:
def extract_features_from_json(data):
    try:
        features = []

        # Timbre
        features += data['lowlevel']['mfcc']['mean']
        features += data['lowlevel']['gfcc']['mean']
        features.append(data['lowlevel']['hfc']['mean'])

        # Tonal - Harmony
        features.append(data['tonal']['chords_changes_rate'])

        # Tonal - Scale (major=0, minor=1)
        scale = data['tonal'].get('key_scale', 'major')
        features.append(1 if scale == 'minor' else 0)

        # Tonal - Pitch salience & dissonance
        features.append(data['lowlevel']['pitch_salience']['mean'])
        features.append(data['lowlevel']['dissonance']['mean'])

        # Rhythm
        features.append(data['rhythm']['bpm'])
        features.append(data['rhythm']['onset_rate'])

        # Spectrum
        features.append(data['lowlevel']['spectral_centroid']['mean'])
        features.append(data['lowlevel']['spectral_complexity']['mean'])
        features.append(data['lowlevel']['spectral_rolloff']['mean'])
        features.append(data['lowlevel']['spectral_flux']['mean'])
        features.append(data['lowlevel']['zerocrossingrate']['mean'])

        # Spectral contrast (6D, not contrast_coeffs)
        features += data['lowlevel']['spectral_contrast_coeffs']['mean']

        # Dynamics
        features.append(data['lowlevel']['average_loudness'])
        features.append(data['lowlevel']['dynamic_complexity'])

        # Rhythm extension
        features.append(data['rhythm']['beats_loudness']['mean'])

        # Energy band shape
        features.append(data['lowlevel']['spectral_energyband_low']['mean'])
        features.append(data['lowlevel']['spectral_energyband_high']['mean'])

        # Harmonic structure
        features.append(data['tonal']['hpcp_entropy']['mean'])
        features.append(data['tonal']['key_strength'])

        # Tonal energy balance
        features.append(data['lowlevel']['spectral_entropy']['mean'])
        features.append(data['lowlevel']['spectral_strongpeak']['mean'])

        return features
    except KeyError as e:
        print(f"Missing key: {e}")
        return None

def build_feature_labels(data_sample):
    labels = []

    labels += [f"mfcc_{i}" for i in range(len(data_sample['lowlevel']['mfcc']['mean']))]
    labels += [f"gfcc_{i}" for i in range(len(data_sample['lowlevel']['gfcc']['mean']))]
    labels += ["hfc"]
    labels += ["chords_changes_rate"]

    labels += ["key_scale"]
    labels += ["pitch_salience"]
    labels += ["dissonance"]
    labels += ["bpm", "onset_rate"]
    labels += ["spectral_centroid", "spectral_complexity", "spectral_rolloff", "spectral_flux", "zerocrossingrate"]
    labels += [f"spectral_contrast_{i}" for i in range(len(data_sample['lowlevel']['spectral_contrast_coeffs']['mean']))]
    labels += ["average_loudness", "dynamic_complexity"]

    labels += ["beats_loudness"]
    labels += ["spectral_energyband_low", "spectral_energyband_high"]
    labels += ["hpcp_entropy", "key_strength"]
    labels += ["spectral_entropy", "spectral_strongpeak"]

    return labels

def process_dataset(root_folder):
    all_features = []
    file_ids = []
    labels_initialized = False
    feature_labels = []

    for subdir, _, files in os.walk(root_folder):
        for file in files:
            if file.endswith(".json"):
                file_path = os.path.join(subdir, file)
                try:
                    with open(file_path, "r") as f:
                        data = json.load(f)

                    features = extract_features_from_json(data)
                    if features is None:
                        continue

                    if not labels_initialized:
                        feature_labels = build_feature_labels(data)
                        labels_initialized = True

                    all_features.append(features)
                    file_ids.append(file.replace('.json', ''))

                except Exception as e:
                    print(f"Failed on {file}: {e}")

    df = pd.DataFrame(all_features, columns=feature_labels)
    df['recordingmbid'] = file_ids
    return df

In [ ]:
# Extract JSON features
folder_path = '../data/acousticbrainz-mediaeval-train'
data_features = process_dataset(folder_path)
#data_features.head()

In [ ]:
merged_df = pd.merge(data_features, data_labeled, on="recordingmbid", how="inner")

# Check results
print(merged_df.head())
print("Shape:", merged_df.shape)
print("Label value counts:\n", merged_df['is_rnb'].value_counts())

merged_df.to_csv("tmp/rnb_features_labeled.csv", index=False)

In [ ]:
df = pd.read_csv("tmp/rnb_features_labeled.csv")

# load data for training
train_data = df.drop(columns=['recordingmbid', 'is_rnb'])
train_labels = df['is_rnb']

In [ ]:
#print(train_data)
rnb = [i for i, label in enumerate(train_labels) if label == 1]
print(rnb)

# Histogram Creation

In [ ]:
histogram_data = [[] for i in range(len(train_data['mfcc_0']))]

for column in train_data:
    for i in range(len(train_data[column])):
        histogram_data[i].append(float(train_data.at[i, column]))

In [ ]:
print(histogram_data[0])

plt.hist(histogram_data[0])
plt.show()

Creating and saving all the histograms will take a fairly long amount of time. If possible, I'd recommend breaking the data down into smaller portions to avoid any errors.

In [ ]:
save_path = os.path.join("CNN_data", "histograms")

# number of histograms processed. I recommend stopping and updating this every 30 minutes
num_histograms = 0

# total_histograms = 111048
total_histograms = len(train_data['mfcc_0'])

for i in range(num_histograms, total_histograms):
    plt.figure()
    histogram = plt.hist(histogram_data[i])
    filename = str(f"histo{i}.png")
    if i in rnb:
        save_label = os.path.join(save_path, "rnb")
    else:
        save_label = os.path.join(save_path, "other")
    save_file = os.path.join(save_label, filename)
    plt.savefig(save_file)
    plt.close()

# CNN Model

In [ ]:
# model definition
CNN_model = models.Sequential([
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),

    layers.Flatten(),
    layers.Dense(64),
    layers.Dense(2)
])

In [ ]:
# model compilation
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
CNN_model.compile(optimizer='adam', metrics=['accuracy'], loss=loss)

# CNN Training

In [ ]:
file_path = os.path.join("CNN_data", "histograms")

histograms = tf.keras.utils.image_dataset_from_directory(file_path,
                                                           color_mode="rgb",
                                                           label_mode="int",
                                                           image_size=(256,256),
                                                           batch_size=100)

In [ ]:
CNN_model.fit(histograms, epochs=5)

In [ ]:
# model summary
CNN_model.summary()

# Save and Load Model

In [ ]:
CNN_model.save("CNN_data/CNN_model.keras")

In [ ]:
CNN_model = tf.keras.models.load_model("CNN_data/CNN_model.keras")

# Model Evaluation

In [ ]:
test_loss, test_acc = CNN_model.evaluate(histograms)
print(f'Test accuracy: {test_acc:.2f}')

# Song Prediction

In [ ]:
# prediction histogram
# change to desired test histogram
test_hist_path = "CNN_data/histograms/other/histo1.png"

img = tf.keras.utils.load_img(test_hist_path, target_size=(256, 256))
test_hist = tf.keras.utils.img_to_array(img)
test_hist = np.expand_dims(test_hist, axis=0)

prediction = CNN_model.predict(test_hist)
prediction = np.argmax(prediction)
print(f'Predicted genre: {prediction}')